### Importamos las librerias necesarias para realizar nuestro proceso de enriquecimiento de datos:

In [2]:
from pyspark.sql.functions import col, substring, concat
from pyspark.sql import SparkSession

### Creacion de tabla `processed_files_reviews_google_silver` y `processed_file_metadata_silver`para mantener el registro de los archivos ya procesados.

In [2]:
spark.sql("CREATE TABLE IF NOT EXISTS processed_files_reviews_google_silver (file_name STRING) USING DELTA")
spark.sql("CREATE TABLE IF NOT EXISTS processed_files_metadata_google_silver (file_name STRING) USING DELTA")

### Aca podes revisar las tablas que mantienen el registro de los archivos ya procesados.

In [13]:
spark.sql("SELECT * FROM processed_files_reviews_google_silver").show()

In [14]:
spark.sql("SELECT * FROM processed_files_metadata_google_silver").show()

### Obtenemos la lista de archivos contenidos en el storage. En este caso los contenidos en la carpeta silver que corresponden a los archivos luego del proceso de ETL.

In [15]:
# Crear una instancia de SparkSession
spark = SparkSession.builder.getOrCreate()

# Especificar el nombre del contenedor y directorio
container_name = "datumtech"
directory_path_reviews = "/silver/GoogleMapssilver/reviews-estados-silver/"

# Obtener la lista de carpetas
adls_files_reviews = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
    .listStatus(spark._jvm.org.apache.hadoop.fs.Path(f"abfss://{container_name}.blob.core.windows.net/{directory_path_reviews}"))

In [16]:
#Directorio de metadata
directory_path_metadata = "/silver/GoogleMapssilver/metadata-sitios-silver/"

# Obtener la lista de carpetas
adls_files_metadata = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
    .listStatus(spark._jvm.org.apache.hadoop.fs.Path(f"abfss://{container_name}.blob.core.windows.net/{directory_path_metadata}"))

### Creamos una variable `new_files` que contiene los archivos que no estan en ADLS.

In [17]:
processed_files_reviews = spark.sql("SELECT file_name FROM processed_files_reviews_google_silver").toPandas()["file_name"].tolist()

new_files_reviews = [file.getPath().getName() for file in adls_files_reviews if file.getPath().getName() not in processed_files_reviews]

In [18]:
processed_files_metadata = spark.sql("SELECT file_name FROM processed_files_reviews_google_silver").toPandas()["file_name"].tolist()

new_files_metadata = [file.getPath().getName() for file in adls_files_metadata if file.getPath().getName() not in processed_files_metadata]

### Podemos ver que archivos no estan procesados aun.

In [19]:
for file in new_files_reviews:
    print(file)

In [20]:
for file in new_files_metadata:
    print(file)

### Cargamos el archivo metadata correspondiente de la tabla silver para poder procesar las tablas reviews-estados-silver.

In [21]:
path_metadata_silver = f"abfss://datumtech@datumlake.dfs.core.windows.net/silver/GoogleMapssilver/metadata-sitios-silver/{new_files_metadata[0]}"
df_metadata = spark.read.format("parquet").load(path_metadata_silver)

In [22]:
df_metadata.show(5)

In [12]:
file_metadata_gold = new_files_metadata[0].rstrip("-silver")

In [23]:
df_metadata.dtypes

###

### Cambiamos el tipo de dato de la columna 'num_of_reviews' para evitar problemas al momento de la ingesta de los datos en la base de datos.

In [24]:
df_metadata = df_metadata.withColumn("num_of_reviews", col("num_of_reviews").cast("int"))

In [ ]:
path_metadata_gold = f"abfss://datumtech@datumlake.dfs.core.windows.net/gold/GoogleMapsgold/metadata-sitios-gold/{file_metadata_gold}-gold"
df_metadata.write.format("parquet").save(path_metadata_gold)

### Creamos la funcion `gold` que realiza todo el proceso de agregacion de valor a la informacion y creacion de tablas secundarias para diferentes analisis y devuelve los dataframes en formato parquet a la tabla gold corrrespondiente a los datos ya procesados y enriquecidos.

In [25]:

def gold(file_reviews):
    
    # Definimos las rutas:
    file_reviews_gold = file_reviews.rstrip("silver")
    path_reviews_silver = f"abfss://datumtech@datumlake.dfs.core.windows.net/silver/GoogleMapssilver/reviews-estados-silver/{file_reviews}"
    path_reviews_gold = f"abfss://datumtech@datumlake.dfs.core.windows.net/gold/GoogleMapsgold/reviews-estados-gold/{file_reviews_gold}gold"

    # Cargamos los archivo desde la tabla silver en el storage.
    df_reviews = spark.read.format("parquet").load(path_reviews_silver)
    
    # La columna "gmap_id" en df_metadata ya contiene valores únicos, filtraremos df_reviews basándonos en la columna "gmap_id":
    df_joined = df_reviews.join(df_metadata, on="gmap_id", how="inner")
    df_reviews = df_joined.select("user_id", "gmap_id", "rating", "text", "date")

    # Cambiamos el tipo de dato de la columna 'rating' para evitar problemas al momento de la ingesta de los datos en la base de datos.
    df_reviews = df_reviews.withColumn("rating", col("rating").cast("int"))

    # Truncamos la columna text a 7500 para evitar problema a la hora de cargar los datos en la base de datos.
    df_reviews = df_reviews.withColumn("text", substring(df_reviews["text"], 1, 7000))

    # Cambiamos el tipo de dato de la columna 'rating' para evitar problemas al momento de la ingesta de los datos en la base de datos.
    df_reviews = df_reviews.withColumn("rating", col("rating").cast("int"))

    # Generamos la columna review_id con valores unicos para ser usada com primary key
    df_reviews = df_reviews.withColumn('review_id', concat(df_reviews['user_id'], df_reviews['gmap_id']))

    # Guardamos los DataFrames df_reviews y df_metadata en la tabla gold correspondiente a los datos procesados y enriquecidos.   
    return df_reviews.write.format("parquet").save(path_reviews_gold)
    

### Aplicamos la funcion `gold` a todos los archivos sin procesar:

In [26]:
for file in new_files_reviews:
    gold(file)

#### Para comprobar que todo se haya ejecutado de manera correcta, podemos traer cualquier archivo de la tabla gold y hacer algunas verificaciones.

In [27]:
path_reviews_gold = f"abfss://datumtech@datumlake.dfs.core.windows.net/gold/GoogleMapsgold/reviews-estados-gold/review-Florida-gold"
df_reviews_gold = spark.read.format("parquet").load(path_reviews_gold)

In [28]:
df_reviews_gold.show(5)

In [29]:
df_reviews_gold.dtypes

### Agregamos a la tabla `processed_files_reviews_google_silver` y `processed_files_metadata_google_silver` los archivos ya procesados.

In [30]:
new_files_df = spark.createDataFrame([(file,) for file in new_files_reviews], ["file_name"])
new_files_df.write.format("delta").mode("append").saveAsTable("processed_files_reviews_google_silver")

In [31]:
new_files_df = spark.createDataFrame([(file,) for file in new_files_metadata], ["file_name"])
new_files_df.write.format("delta").mode("append").saveAsTable("processed_files_metadata_google_silver")

### Podemos verificar que, efectivamente esten registrados los archivos ya procesados.

In [27]:
spark.sql("SELECT * FROM processed_files_reviews_google_silver ").show()

In [28]:
spark.sql("SELECT * FROM processed_files_metadata_google_silver ").show()

### Podemos verificar si hay nulos en algun archivo en la tabla silver.

In [32]:
# Funcion para el conteo de nulos del dataframe.
'''def null_counts (df):
    counts = df.select([sum(col(c).isNull().cast("integer")).alias(c) for c in df.columns])
    return counts.show()'''

In [ ]:
# Vemos que columnas poseen nulos y en que cantidad.
#nulls = null_counts(df)